In [1]:
from datetime import datetime, timedelta
from calendar_view.core import data
from calendar_view.config import style
from calendar_view.core.config import CalendarConfig
from calendar_view.calendar import Calendar
from calendar_view.core.event import Event
from calendar_view.core.event import EventStyles
from calendar_view.core.event import EventStyle
import requests

In [2]:
learning_maps = (
    ('SP Networking Routing Infrastructure', '1614365762802003wkIr'),
    ('Cisco Meraki', '1614365278184001wl7w'),
    ('IPv6', '1669112925198001TUYV'),
    ('ACI', '1614365003478001MP4u'),
    ('NXOS', '1614365003478002MIBb'),
    ('Compute', '1614365003478003MKRH'),
    ('SD-Access', '1614365278184010wchb'),
    ('SD-WAN', '1614365278184011wdNo'),
    ('Optical Infrastructure and Optics', '1614365762802006w2A9'),
    ('SP Automation and Orchestration', '1614365762802005wrv8'),
    ('Sustainability', '1654445964194002bNUB'),
    ('Collaboration Applications and Integrations', '1614364767910002wlvj')
)

In [3]:
def get_sessions(learning_map: str):
    '''
    Take a learning map and returns the associated sessions in a json object.
    '''

    url = "https://events.rainfocus.com/api/search"

    payload =   {'search.learningmap': learning_map,
                'type': 'session'}

    headers = {
    'rfapiprofileid': '0wnUkT1BBZK3JR2t3yc5huPwNQCS8C3n',
    'referer': 'https://www.ciscolive.com/'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

In [4]:
class Session():

    def __init__(self, session_json):
        self.type = session_json['type']
        try:
            self.start = datetime.strptime(session_json['times'][0]['utcStartTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
            self.end = datetime.strptime(session_json['times'][0]['utcEndTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
        except KeyError:
            self.start = 'Null'
            self.end = 'Null'
        self.name = session_json['title']
        self.id = session_json['code']
        match int(self.id.split('-')[1][0]):
            case 1: self.level = 'Introductory'
            case 2: self.level = 'Intermediate'
            case 3: self.level = 'Advanced'
            case 4: self.level = 'General'
        
    def __str__(self):
        return self.name

In [5]:
for LEARNING_MAP_NAME, LEARNING_MAP in learning_maps:
    print(f'{LEARNING_MAP_NAME = }, {LEARNING_MAP = }')

    sessions_json = get_sessions(LEARNING_MAP)

    sessions = []

    for session in sessions_json['sectionList'][0]['items']:
        sessions.append(Session(session))

    style.hour_height = 500
    style.day_width = 1500
    style.event_notes_color = '#0D274D'
    style.title_font = style.image_font(250)
    style.hour_number_font = style.image_font(50)
    style.day_of_week_font = style.image_font(150)
    style.event_title_font = style.image_font(80)
    style.event_notes_font = style.image_font(60)


    config = data.CalendarConfig(
        lang='en',
        title=LEARNING_MAP_NAME,
        dates='2023-02-06 - 2023-02-10',
        show_date=True,
        mode='working_hours',
        title_vertical_align='top',
    )

    events = []

    for session in sessions:

        if session.type != 'Walk-in Lab':
            match session.level:
                    case 'Introductory': 
                        color = EventStyle(event_border=(116, 191, 75, 240), event_fill=(116, 191, 75, 180))
                    case 'Intermediate': 
                        color = EventStyle(event_border=(251, 171, 44, 240), event_fill=(251, 171, 44, 180))
                    case 'Advanced': 
                        color = EventStyle(event_border=(227, 36, 27, 240), event_fill=(227, 36, 27, 180))
                    case 'General': 
                        color = EventStyle(event_border=(0, 188, 235, 240), event_fill=(0, 188, 235, 180))
                    
            events.append(Event(day=session.start.strftime('%Y-%m-%d'), 
                                start=session.start.strftime('%H:%M'), 
                                end=session.end.strftime('%H:%M'), 
                                title=session.id,
                                notes=session.name,
                                style=color))

    calendar = Calendar.build(config)
    calendar.add_events(events)
    calendar.save('./learning_maps/' + LEARNING_MAP_NAME + ".png")

LEARNING_MAP_NAME = 'SP Networking Routing Infrastructure', LEARNING_MAP = '1614365762802003wkIr'
LEARNING_MAP_NAME = 'Cisco Meraki', LEARNING_MAP = '1614365278184001wl7w'
LEARNING_MAP_NAME = 'IPv6', LEARNING_MAP = '1669112925198001TUYV'
LEARNING_MAP_NAME = 'ACI', LEARNING_MAP = '1614365003478001MP4u'
LEARNING_MAP_NAME = 'NXOS', LEARNING_MAP = '1614365003478002MIBb'
LEARNING_MAP_NAME = 'Compute', LEARNING_MAP = '1614365003478003MKRH'
LEARNING_MAP_NAME = 'SD-Access', LEARNING_MAP = '1614365278184010wchb'
LEARNING_MAP_NAME = 'SD-WAN', LEARNING_MAP = '1614365278184011wdNo'
LEARNING_MAP_NAME = 'Optical Infrastructure and Optics', LEARNING_MAP = '1614365762802006w2A9'
LEARNING_MAP_NAME = 'SP Automation and Orchestration', LEARNING_MAP = '1614365762802005wrv8'
LEARNING_MAP_NAME = 'Sustainability', LEARNING_MAP = '1654445964194002bNUB'
LEARNING_MAP_NAME = 'Collaboration Applications and Integrations', LEARNING_MAP = '1614364767910002wlvj'
